# Assignment - 2: Data Representation and Point Cloud Operations

Team Name: Robo-Knights

Roll number:  2019111007, 2019112002

# Instructions

- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment.
- Save all your results in ```results/<question_number>/<sub_topic_number>/```
- The **References** section provides you with important resources to solve the assignment.
- Make sure your code is modular since you may need to reuse parts for future assignments.
- Answer the descriptive questions in your own words with context & clarity. Do not copy answers from online resources or lecture notes.
- The **deadline** for this assignment is on 26/09/2021 at 11:55pm. Please note that there will be no extensions.
- Plagiarism is **strictly prohibited**.

# Submission Instructions

1. Make sure your code runs without any errors after reinitializing the kernel and removing all saved variables.
2. After completing your code and saving your results, zip the folder with name as ``Team_<team_name>_MR2021_Assignment_<assignment_number>.zip``

In [1]:
# !pip install numpy
# !pip install matplotlib
# !pip install opencv-python
# !pip install python-utils
# !pip show requests
# !pip install pynput
import pickle
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os
import glob
from open3d import *
from os import listdir
from os.path import isfile, join
from PIL import Image
import math
from numpy.linalg import inv
from pynput.keyboard import Key, Listener
import copy

INFO - 2021-09-28 20:55:27,548 - utils - NumExpr defaulting to 8 threads.


# Introduction to types of Transformations and Homogeneous coordinates

In robotics applications, it is inevitable to keep track of the frames of multiple objects/worlds. These frames can be transformations from one coordinate frame to the other. **Homogeneous coordinates** help in keeping track of various coordinate frames and allow performing composition of various transforms. We will first try to understand between types of transformations and their invariant properties.
1. What is the difference between Affine, Similarity, and Euclidean transform? What are the invariant properities of each type of transform?
2. Watch this [video](https://www.youtube.com/watch?v=PvEl63t-opM) to briefly understand homogeneous coordinates. What are points at infinity? What type of transformation can you apply to transform a point from infinity to a point that is not at infinity? 
3. Using homogeneous coordinates we can represent different types of transformation as point transforms vs. frame transforms. Concatenation of transforms (whether you post multiply transformation matrices or pre-multiply transformation matrices) depends on the problem and how you are viewing it. Try to understand the difference between frame vs. point transformations from this [video](https://youtu.be/Za7Sdegf8m8?t=1834). Let's assume that our camera and world frames are coinciding with each other. We need to estimate the camera to world **frame** transformation matrix after applying the transformations defined below in terms of $T_i$.We apply **frame** transform to move the camera in the world in the following order:
    1. $T_1$ from the camera coordinate frame.
    2. $T_2$ from the world coordinate frame.
    3. $T_3$ from the world coordinate frame.
    4. $T_4$ from the camera coordinate frame.
    5. $T_5$ from the camera coordinate frame.


1. 
    a. Affine transform preserves lines (meaning that it sends points to points, lines to lines, planes to planes, and so on) and parallelism i.e., all points lying on a line initially still lie on a line after transformation and it preserves ratios and distances (along parallel directions) but not necessarily distances and euler angles. Eg: Reflection, rotation, shear, translation, etc.  
    
    b. Euclidean transform preserves distances and angles. It can only change orientation and position of a figure in space. The size of the figure remains same. Eg: Rotation, translation, reflection. Any Euclidean transformation is an affine transformation.
    
    c. Let C be a square nonsingular matrix having the same order as the matrix A. We say that the matrices A and $C^{−1}AC$ are similar, and the transformation from A to $C^{−1}AC$ is called a similarity transformation. Moreover, we say that the two matrices are unitarily similar if C is unitary. Similarity transformations transform objects in space to similar objects.The determinant of the similarity transformation of a matrix is equal to the determinant of the original matrix. Two similar matrices share the same spectrum and the same characteristic polynomial.Properties: 
    1. Distinct points are mapped to distinct points. 
    2. Each point 𝑃′ in the plane has a pre-image. 
    3. A similarity transformation sends lines to lines, rays to rays, line segments to line segments, and parallel lines to parallel lines. 
    4. A similarity transformation sends angles to angles of equal measure. 
    5. A similarity transformation maps a circle of radius 𝑅 to a circle of radius 𝑟𝑅, where 𝑟 is the scale factor of the similarity transformation

2. Points at infinity are the the points that are at a very large distance from camera. Informally defined as the limit of a point that moves in that direction away from the origin. point at infinity corresponding to the direction of the line ax + by = 0. Points at infinity can be expressed in homogeneous by setting the last coordinate to 0. Points at infinity expressed in homogeneous coordinates preserves the direction of the point. To convert from point at infinity to a finite point,we set the last coordinate to a finite value. A point at infinity can be shown to transform to a finite point using perspective projection.  
One possible transformation matrix is: [[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, r ,1]], point at infinity = [x, y, z, 0]. Here r is a constant and r and z are non-zero.

3. Assuming $T_i$ is the initial transformation matrix for a camera.  Camera to world frame transformation matrix: $(T_3T_2T_iT_1T_4T_5)^{-1}$
    1. $T_1$ from the camera coordinate frame: Since the tranformation is done in current frame we post multiply. T = $T_i$$T_1$
    2. $T_2$ from the world coordinate frame: Since the tranformation is done in world frame we pre multiply.  T = $T_2$$T_i$$T_1$
    3. $T_3$ from the world coordinate frame: Since the tranformation is done in world frame we pre multiply.  T = $T_3$$T_2$$T_i$$T_1$
    4. $T_4$ from the camera coordinate frame: Since the tranformation is done in current frame we post multiply. T = $T_3$$T_2$$T_i$$T_1$$T_4$
    5. $T_5$ from the camera coordinate frame: Since the tranformation is done in current frame we post multiply. T = $T_3$$T_2$$T_i$$T_1$$T_4$$T_5$

But since we need transformation from the camera to world frame we take inverse of T. Therefore final transformation matrix is <br/>
$(T_3T_2T_iT_1T_4T_5)^{-1}$ =
$T_5^{-1}$*$T_4^{-1}$*$T_1^{-1}$*$T_i^{-1}$*$T_2^{-1}$$T_3^{-1}$ 

If $T_i$ is not the initial transformation matrix for a camera, the final transformation matrix is : $T_5^{-1}$*$T_4^{-1}$*$T_1^{-1}$*$T_2^{-1}$*$T_3^{-1}$ 

# Visualise the Data

Point clouds are a collection of points that represent a 3D shape or feature. Each point has its own set of X, Y and Z coordinates and in some cases additional attributes. A popular way to obtain this is by photogrammetry, though here we will use LiDAR data.

LiDAR is a remote sensing process which collects measurements used to create 3D models and maps of objects and environments. Using ultraviolet, visible, or near-infrared light, LiDAR gauges spatial relationships and shapes by measuring the time it takes for signals to bounce off objects and return to the scanner.

1. Download the data from [here](https://iiitaphyd-my.sharepoint.com/:f:/g/personal/venkata_surya_students_iiit_ac_in/EnYAMaTVIhJItzKYqtahE30BRKB6p6UfHN3TyJzvo6Mw0g?e=PegWds). It contains the LIDAR sensor output and odometry information per frame.

    The .bin files contain the 3D point cloud captured by the LIDAR in this format - x, y, z, and reflectance. 

    The odometry information is given in the `odometry.txt` file, which is a 12 element vector. Reshape each of the first 77 rows to a 3x4 matrix to obtain the pose.
    

2. Obtain the point cloud from this and visualise for 1-2 frames.

In [2]:
# reading the data from odometry.txt
# 1
def readData(filename):
    data = np.loadtxt(filename)
    return data 

# reading the bin files
def readPointCloud(filename):
    pcl = np.fromfile(filename, dtype=np.float32,count=-1)
    pcl = pcl.reshape([-1,4])
    return pcl

# Obtaining the transformation matrix
def getTransformationMat(odometry ,i):
    T = odometry[i][:]
    T = np.reshape(T, (3,4))
    b = np.array([0,0,0,1])
    T = np.vstack ((T,b)) 
    return T

# Displaying point cloud from the given list
def displayPointCloud(pc):
    pcd_dis = o3d.geometry.PointCloud()  
    len_ = len(pc)
    for nn in range(len_):
        pcd = o3d.geometry.PointCloud()
        each_pc = pc[nn].T[:, :3]
        pcd.points = o3d.utility.Vector3dVector(each_pc)
        pcd_dis += pcd
    pcd_dis = pcd_dis.voxel_down_sample(voxel_size = 1)
    o3d.visualization.draw_geometries([pcd_dis])

In [3]:
directory_lidar = './data/LiDAR/'
directory_odometry = './data/odometry.txt'

odometry_data = readData(directory_odometry)
odometry_data = odometry_data[:77][:]

# Saving the lidar file names 
lidarFiles = [f for f in listdir(directory_lidar) if isfile(join(directory_lidar, f))]

lidarFiles = sorted(lidarFiles)

pc  = []
pc_ = []
pc__ = []
# 2
# Iterating through the 77 bin files and saving the point for displaying
for f in lidarFiles: 
    f_data = readPointCloud(directory_lidar + f)
    x = np.ones(f_data.shape[0])
    # Replacing the reflectance values with 1 as they are not needed.
    f_data[:,3] = x 
    f_data = f_data.T
    pc.append(f_data)
    
pc_.append(pc[0]) 
pc__.append(pc[33])
# Visualizing 1th frame
displayPointCloud(pc_)
# Visualizing 34th frame
displayPointCloud(pc__)

# Transform 

The point cloud obtained is with respect to the LiDAR frame. The poses however, are in the camera frame. If we want to combine the point clouds from various frames, we need to bring them to the camera frame. 

1. Refer to the image below and apply the required transformation to the point cloud. 

2. Then, register all point clouds into a common reference frame and visualise it (Open3D). It is helpful to use homogeneous coordinates to keep track of the different frames.

3. Write a function to transform the registered point cloud from the world to the $i^{th}$ camera frame, wherein $i$ is the input to the function.

4. \[Bonus\] Move around in the registered point cloud using arrow keys like you would do in a game. For this you will have to regularly transform the entire registered world to your current camera frame and visualize repeatedly. You may choose to avoid visualizing points that are behind the camera in this case as they are not visible from the scene. You may also visualize points at a max depth to make the process easier.

![](./img/transform.png)

In [4]:
# 1
# T_lidar_camera = [[Xl.Xc, Yl.Xc, Zl.Xc, 0], [Xl.Yc, Yl.Yc, Zl.Yc,0], [Xl.Zc, Yl.Zc, Zl.Zc,0], [0,0,0,1]]
T_lidar_camera = np.array([[0,-1,0,0],[0,0,-1,0],[1,0,0,0],[0,0,0,1]]) 
# T_camera_world = inv(T_lidar_camera)
T_camera_world = np.array([[0,0,1,0],[-1,0,0,0],[0,-1,0,0],[0,0,0,1]])

points_in_world = []
point_in_camera = []
points_in_world_world = []
# Transform the point cloud from the Lidar’s frame to the Camera’s frame
for i in range(77): 
    point_in_camera.append(T_lidar_camera@pc[i])
    
# 2
# Transform the point cloud from the Camera’s frame to the Common frame 
# Apply the global transformation obtained from the .txt file.
# World frame is considered assuming the given transformation takes the points form camera frame to the world
for i in range(77):
#   Common frame
    points_in_world.append(T_camera_world@(getTransformationMat(odometry_data ,i)@point_in_camera[i])) 
#   World frame
    points_in_world_world.append((getTransformationMat(odometry_data ,i)@point_in_camera[i])) 
    
displayPointCloud(points_in_world)
# displayPointCloud(points_in_world_world)

In [5]:
# 3
# Transforming from common frame to ith camera frame
def transformation_ith_cam(points, cam_num):
    points_given_cam = []
    for i in range(77):
        points_given_cam.append(inv(T_camera_world@(getTransformationMat(odometry_data ,cam_num)))@points[i])
#     displaying the transformed point cloud    
    displayPointCloud(points_given_cam)    

In [6]:
# Visualizing form the ith_frame the registered point cloud
ith_frame = 4
transformation_ith_cam(points_in_world, ith_frame - 1)

In [16]:
# 4 Bonus Obtaining the transformed point cloud in the required camera frame
def transform_to_given_camera(points, cam_num):
    point_in_reqframe = []
    for i in range(77):
        point_in_reqframe.append(inv(T_camera_world@(getTransformationMat(odometry_data ,cam_num)))@points[i])
    pcd_dis = o3d.geometry.PointCloud()  
    len_ = len(point_in_reqframe)
    for nn in range(len_):
        pcd = o3d.geometry.PointCloud()
        each_pc = point_in_reqframe[nn].T[:, :3]
        pcd.points = o3d.utility.Vector3dVector(each_pc)
        pcd_dis += pcd
    pcd_dis = pcd_dis.voxel_down_sample(voxel_size = 1)
    return pcd_dis   

In [17]:
def custom_draw_geometry_with_key_callback(pcd):
    
    
    def move_to_next_frame(vis):
        global current_frame
        global current_trans 
        current_frame = (current_frame + 1)%77
#       Obtaining the required transformation matrix required to transform the points from common frame to current camera frame 
        mat = inv(T_camera_world@getTransformationMat(odometry_data ,current_frame))
#       Taking the current point first to common frame and then to the required camera frame
        net_trans = mat@current_trans
#       Setting the view control to only display the points in front of the camera
        ctr = vis.get_view_control()
        ctr.change_field_of_view(step=90)
        ctr.set_front([ -0.015120344908731674, 0.044412925228585441, -0.99889882733061675 ])
        ctr.set_lookat([ -9.5100493742385073, -3.7686547525759422, 26.032425470440081 ])
        ctr.set_up([ -0.0361140904310902, -0.99838545923018884, -0.043843440445063948 ])
        ctr.set_zoom(0.25999999999999995)
        pcd.transform(net_trans)
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        current_trans = inv(mat)
        return False
    
    def move_to_previous_frame(vis):
        global current_frame
        global current_trans 
        current_frame = (current_frame - 1)%77
        mat = inv(T_camera_world@getTransformationMat(odometry_data ,current_frame))
        
        net_trans = mat@current_trans
        ctr = vis.get_view_control()
        ctr.change_field_of_view(step=90)
        ctr.set_front([ -0.015120344908731674, 0.044412925228585441, -0.99889882733061675 ])
        ctr.set_lookat([ -9.5100493742385073, -3.7686547525759422, 26.032425470440081 ])
        ctr.set_up([ -0.0361140904310902, -0.99838545923018884, -0.043843440445063948 ])
        ctr.set_zoom(0.25999999999999995)
        pcd.transform(net_trans)
        vis.update_geometry(pcd)
        vis.poll_events()
        vis.update_renderer()
        current_trans = inv(mat)
        return False
    
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size = 30, origin = [-9.5,0,-20])
    
    key_to_callback = {}
    key_to_callback[ord("D")] = move_to_next_frame
    key_to_callback[ord("A")] = move_to_previous_frame
    o3d.visualization.draw_geometries_with_key_callbacks([pcd], key_to_callback)

In [18]:
# Enter the starting frame 
current_frame = 50
pointCloud_obtained = transform_to_given_camera(points_in_world, current_frame)
current_trans = T_camera_world@getTransformationMat(odometry_data , current_frame)
print("   Press 'D' to move to next frame")
print("   Press 'A' to move to previous frame")
custom_draw_geometry_with_key_callback(pointCloud_obtained)

   Press 'D' to move to next frame
   Press 'A' to move to previous frame


Find the link for the recorded animated video: https://drive.google.com/drive/folders/1jBnCNjmx7yScjUowzy7RjjBTHZ5y2Awv?usp=sharing

# Occupancy Grid
Occupancy grid maps are discrete fine grain grid maps. These maps can be either 2-D or 3-D. Each cell in the occupancy grid map contains information on the physical objects present in the corresponding space. Since these maps shed light on what parts of the environment are occupied, and what is not, they are really useful for path planning and navigation.

Occupancy grid maps are probabilistic in nature due to noisy measurements. Each cell can have three states: Occupied, unoccupied, and unknown. For the purpose of this assignment, you can ignore the unknown and work in a binary setting where 1 is occupied and 0 is unoccupied.

1. The task here is to create an occupancy map for each LiDAR scan. You do not need to apply bayesian update rules here, just keep it simple. 

2. Now, using the *registered* point cloud, generate occupancy maps for each frame. What difference do you expect to see between the two methods?

You can mark a cell as occupied based on a threshold of how many different z values are there for a particular (x,y) cell.

In [10]:
# 1
def create_occupancy_grid_LiDAR_i(points, threshold, occ):
    len_ = len(points)
    dimensions = np.zeros((len_,4), dtype='int64')
    for i in range(len_): 
#       Since the occupancy grid is basically an image with discrete integer indices, we will round off the Lidar’s measurements to integer values.
        points_rounded = points[i].astype('int64')
        points_rounded = points_rounded.T[:, :3]
        
#       Next, we find the max and min values for the x and y coordinates in order to determine the dimensions of the occupancy grid.
        dimensions[i,0] = np.amax(points_rounded[:,0])
        dimensions[i,1] = np.amin(points_rounded[:,0])
        dimensions[i,2] = np.amax(points_rounded[:,1])
        dimensions[i,3] = np.amin(points_rounded[:,1])
        occupancy_grid = np.zeros((abs(dimensions[i,0] - dimensions[i,1]) + 1, abs(dimensions[i,2] - dimensions[i,3]) + 1))
        
#       Since we want to generate different values of z for a particular (x,y), we need to remove the duplicate coordinates.
        points_rounded = np.unique(points_rounded, axis=0)
    
#    After removing duplicates count the occurrence of points which fall in the same cell but counting the different values of z for a pair in the array
        for nn in range(points_rounded[:,1].size):
            occupancy_grid[points_rounded[nn,0] - dimensions[i,1], points_rounded[nn,1] - dimensions[i,3]] += 1
        if occ == 1:
#             free
#       For obtaining the free map we normalize so as to set a proper threshold that would give us the free area in the map.
            occupancy_grid = occupancy_grid/points_rounded[:,1].size
            image_path = str(i)+"_free.png"
    
        else:
#             occupied
            image_path = str(i)+"_occupied.png"
    
#       After getting the count of multiple z we mark the cell as an obstacle only if the count of z values for that cell is greater than threshold else it will be considered as empty.
        occupancy_grid = (occupancy_grid > threshold)
#       The results are saved using the PIL library.
        save_to_path = './results/Question_4_1/'
        path_final = save_to_path + image_path
        np.asarray(occupancy_grid)
        occupancy_grid = Image.fromarray(occupancy_grid)
        occupancy_grid.save(path_final, 'PNG')
        

In [11]:
# occupied
create_occupancy_grid_LiDAR_i(pc, 1, 0)
# free
create_occupancy_grid_LiDAR_i(pc, 0.00003, 1)

In [12]:
# 2
def create_occupancy_grid_LiDAR_ii(points, threshold, occ):
    # Finding the dimension of the grid
    all_points = []
    dimensions = np.zeros((1,4), dtype='int64')
    len_ = len(points)
    
    for i in range(len_): 
#       Since the occupancy grid is basically an image with discrete integer indices, we will round off the Lidar’s measurements to integer values.
        points_rounded = points[i].astype('int64')
        points_rounded = points_rounded.T[:, :3]
#       Since we want to generate different values of z for a particular (x,y), we need to remove the duplicate coordinates.
        points_rounded = np.unique(points_rounded, axis=0)
        all_points.append(points_rounded)
    
    final_array = all_points[0]
    
    for i in range(len_ - 1): 
        final_array = np.concatenate((final_array, all_points[i+1]), axis=0)  
        
#   Since we want to generate different values of z for a particular (x,y), we need to remove the duplicate coordinates.    
    final_array_rounded = np.unique(final_array, axis=0)
    
#   Next, we find the max and min values for the x and y coordinates in order to determine the dimensions of the occupancy grid.    
    dimensions[0,0] = np.amax(final_array_rounded[:,0])
    dimensions[0,1] = np.amin(final_array_rounded[:,0])
    dimensions[0,2] = np.amax(final_array_rounded[:,1])
    dimensions[0,3] = np.amin(final_array_rounded[:,1])
    occupancy_grid = np.zeros((abs(dimensions[0,0] - dimensions[0,1]) + 1, abs(dimensions[0,2] - dimensions[0,3]) + 1))
    
#    After removing duplicates count the occurrence of points which fall in the same cell but counting the different values of z for a pair in the array
    for nn in range(final_array_rounded[:,1].size):
        occupancy_grid[final_array_rounded[nn,0] - dimensions[0,1], final_array_rounded[nn,1] - dimensions[0,3]] += 1

#   After getting the count of multiple z we mark the cell as an obstacle only if the count of z values for that cell is greater than threshold else it will be considered as empty.    
    if occ == 1:
#       For obtaining the free map we normalize so as to set a proper threshold that would give us the free area in the map.
        occupancy_grid = occupancy_grid/final_array_rounded[:,1].size
        image_path = "Final_4_ii_free"+".png"    
    else:
        image_path = "Final_4_ii_occupied"+".png"
        
    occupancy_grid = (occupancy_grid > threshold)
    
#   The results are saved using the PIL library.    
    save_to_path = './results/Question_4_2/'
#   image_path = "Final_4_ii"+".png"
    path_final = save_to_path + image_path
    np.asarray(occupancy_grid)
    occupancy_grid = Image.fromarray(occupancy_grid)
    occupancy_grid.save(path_final, 'PNG')

In [13]:
# occupied
create_occupancy_grid_LiDAR_ii(points_in_world, 2, 0)
# free 
create_occupancy_grid_LiDAR_ii(points_in_world, 0.00003, 1)

Differences Observed:
1. In (1) the array we took is only points corresponding to the particular timestamp calculation whereas in (2) we took the array of points which includes all the 77 time stamps together.
2. The visualized point cloud in case (2) is more dense than in case (1) since we visualize the all the 77 frames in a single window in case (2) and only one frame is visualized in part (1). The threshold taken in case (1) is 1 whereas (2) we will increase the threshold for better estimate of the obstacle at that particular position. This is because since we are joining points form different point clouds the probability of error increases. So to get a better map from noisy measurements we increase the threshold value.